# 0. Dependencies


In [ ]:
import os
import bs4
import math
import json
import requests
import itertools

import regex as re
import numpy as np
import pandas as pd

from datetime import date
from getpass import getpass
from bs4 import BeautifulSoup
from xgboost import XGBRegressor

from dateutil.relativedelta import relativedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Obtendo o token 'x-funds-nonce'
token_url = 'https://www.fundsexplorer.com.br/wp-content/themes/fundsexplorer/dist/frontend.min.js'
response = requests.get(token_url)
funds_explorer_token = re.search(r'(?<=\"x-funds-nonce\":\s*\").*?(?=\")', response.text).group()

In [ ]:
# Variáveis globais
start_date = '01-01-2021'
end_date = '01-01-2024'

# 1. FIIs listados na B3

In [ ]:
def get_all_funds():

    # Fazendo requisição à API com o token obtido
    url = 'https://www.fundsexplorer.com.br/wp-json/funds/v1/get-ranking'
    response = requests.get(url, headers={
        'accept': 'application/json, text/plain, */*',
        'x-funds-nonce': funds_explorer_token
    })

    # Convertendo a resposta JSON para DataFrame
    df = pd.read_json(response.json())

    return df

# Exibindo o DataFrame
df_funds = get_all_funds()
df_funds

,post_id,ticker,dividendo,yeld,media_yield_3m,soma_yield_3m,media_yield_6m,soma_yield_6m,media_yield_12m,soma_yield_12m,...,patrimonio,pvp,p_vpa,post_title,ativos,volatility,numero_cotista,tx_gestao,tx_admin,tx_performance
0,2555,AAGR11,1.0636,1.07,1.1400,3.42,1.3850,5.54,1.3850,5.54,...,0.000000e+00,0.0000,NaN,AAGR11,,58.782301975493,0,,,
1,495,AAZQ11,0.1100,1.47,1.3067,3.92,1.3033,7.82,1.3008,15.61,...,2.086440e+08,0.8940,0.894009,AAZQ11,,17.836355311082,27954,,,
2,238,ABCP11,0.3500,0.50,0.6100,1.83,0.6950,4.17,0.7600,9.12,...,4.778136e+08,0.6642,0.664236,ABCP11,1,14.918635937379,16357,,,
3,177,AFHI11,0.9500,0.97,0.9833,2.95,0.9767,5.86,1.0183,12.22,...,3.839856e+08,1.0174,1.017383,AFHI11,0,6.921895944248,38421,,,
4,553,AGRX11,0.1300,1.33,1.3467,4.04,1.2617,7.57,1.2983,15.58,...,0.000000e+00,0.0000,NaN,AGRX11,0,15.67576760692,0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,537,YUFI11B,0.6331,0.00,0.0000,0.00,0.0000,0.00,0.0000,0.00,...,0.000000e+00,0.0000,NaN,YUFI11B,,,0,,,
479,3486,ZAGH11,0.1100,0.92,0.3067,0.92,0.3067,0.92,0.3067,0.92,...,4.667184e+07,0.1195,NaN,ZAGH11,,390.42346644832,120,,,
480,2972,ZAVC11,0.1100,0.00,0.0000,0.00,0.0000,0.00,0.0000,0.00,...,3.083557e+07,1.0050,NaN,ZAVC11,,8.340380435713,84,,,
481,1866,ZAVI11,1.0700,0.94,0.9433,2.83,0.9733,5.84,0.9492,11.39,...,1.555315e+08,0.7693,NaN,ZAVI11,11,27.165252081362,4392,,,


# 2. Dividendos pagos

In [ ]:
def get_dividends(ticker_list):
    result = pd.DataFrame()

    for index, ticker in enumerate(ticker_list):
        print(f'Obtendo dividendos: {index+1}/{len(ticker_list)}')

        url = f'https://www.fundsexplorer.com.br/wp-json/funds/v1/fund/{ticker}/incomes'

        headers = {
            'accept': 'application/json, text/plain, */*',
            'x-funds-nonce': funds_explorer_token
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            df = pd.read_json(response.json())
            result = pd.concat([result, df], ignore_index=True)
        else:
            print(f"Não foi possível obter os dividendos para o ticker {ticker}.")

    return result

df_dividends = get_dividends(list(df_funds['ticker']))

Obtendo dividendos: 1/483
Obtendo dividendos: 2/483
Obtendo dividendos: 3/483
Obtendo dividendos: 4/483
Obtendo dividendos: 5/483
Obtendo dividendos: 6/483
Obtendo dividendos: 7/483
Obtendo dividendos: 8/483
Obtendo dividendos: 9/483
Obtendo dividendos: 10/483
Obtendo dividendos: 11/483
Obtendo dividendos: 12/483
Obtendo dividendos: 13/483
Obtendo dividendos: 14/483
Obtendo dividendos: 15/483
Obtendo dividendos: 16/483
Obtendo dividendos: 17/483
Obtendo dividendos: 18/483
Obtendo dividendos: 19/483
Obtendo dividendos: 20/483
Obtendo dividendos: 21/483
Obtendo dividendos: 22/483
Obtendo dividendos: 23/483
Obtendo dividendos: 24/483
Obtendo dividendos: 25/483
Obtendo dividendos: 26/483
Obtendo dividendos: 27/483
Obtendo dividendos: 28/483
Obtendo dividendos: 29/483
Obtendo dividendos: 30/483
Obtendo dividendos: 31/483
Obtendo dividendos: 32/483
Obtendo dividendos: 33/483
Obtendo dividendos: 34/483
Obtendo dividendos: 35/483
Obtendo dividendos: 36/483
Obtendo dividendos: 37/483
Obtendo di

In [ ]:
df_dividends

,id,ticker,created_at,updated_at,valor,tipo,data_pagamento,data_base,yeld,rentabilidade,...,mes,ano,rentabilidade_mes,media_yield_3m,media_yield_6m,media_yield_12m,soma_yield_3m,soma_yield_6m,soma_yield_12m,soma_yield_ano_corrente
0,8152428,AAGR11,2024-07-14 19:03:02,2024-07-14 19:03:02,1.0636,Rendimento,2024-07-15,2024-07-08,1.07,10.0738,...,7.0,2024.0,3.0633,1.1400,1.385,1.385,3.42,5.54,5.54,5.54
1,8152427,AAGR11,2024-07-14 19:03:02,2024-07-14 19:03:02,1.1251,Rendimento,2024-06-14,2024-06-07,1.15,6.8022,...,6.0,2024.0,0.1370,1.1767,1.490,1.490,3.53,4.47,4.47,4.47
2,8152426,AAGR11,2024-07-14 19:03:02,2024-07-14 19:03:02,1.1888,Rendimento,2024-05-15,2024-05-08,1.20,6.6561,...,5.0,2024.0,12.4381,1.6600,1.660,1.660,3.32,3.32,3.32,3.32
3,8152425,AAGR11,2024-07-14 19:03:02,2024-07-14 19:03:02,1.0455,Rendimento,2024-04-15,2024-04-08,1.18,-5.1424,...,4.0,2024.0,-5.1424,2.1200,2.120,2.120,2.12,2.12,2.12,2.12
4,8152424,AAGR11,2024-07-14 19:03:02,2024-07-14 19:03:02,0.8900,Rendimento,2024-03-15,2024-03-08,0.94,NaN,...,3.0,2024.0,0.0000,0.9400,0.940,0.940,0.94,0.94,0.94,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20522,8152474,ZAVI11,2024-07-14 19:03:09,2024-07-14 19:03:09,1.2180,Rendimento,2023-01-13,2022-12-29,0.94,0.9400,...,1.0,2023.0,0.9400,0.9250,0.925,0.925,1.85,1.85,1.85,0.94
20523,8152473,ZAVI11,2024-07-14 19:03:09,2024-07-14 19:03:09,1.1800,Rendimento,2022-12-14,2022-11-30,0.91,NaN,...,12.0,2022.0,0.0000,0.9100,0.910,0.910,0.91,0.91,0.91,0.91
20524,8152472,ZAVI11,2024-07-14 19:03:09,2024-07-14 19:03:09,1.1400,Rendimento,2022-11-16,2022-10-31,0.00,NaN,...,11.0,2022.0,0.0000,0.0000,0.000,0.000,0.00,0.00,0.00,0.00
20525,8162853,ZIFI11,2024-07-14 19:09:33,2024-07-14 19:09:33,3.3050,Rendimento,2022-07-14,2022-06-30,0.00,NaN,...,7.0,2022.0,0.0000,0.0000,0.000,0.000,0.00,0.00,0.00,0.00


# 3. Preços de fechamento

In [ ]:
def get_close(ticker_list):

    result = pd.DataFrame()

    for index, ticker in enumerate(ticker_list):

        print(f'Obtendo preços de fechamento: {index+1}/{len(ticker_list)}')

        response = requests.get(f'https://fii-api.infomoney.com.br/api/v1/fii/cotacao/historico/grafico?Ticker={ticker}&DataInicio={start_date}&DataFim={end_date}')

        if not str(response.content) == "b''":

            json_response = json.loads(response.content)

            if 'errors' in json_response or response.status_code != 200:
                print(f"Não foi possível obter os preços de fechamento para o ticker {ticker}.")
            else:
                df = pd.read_json(json.dumps(json_response['dataValor']))
                df['ticker'] = ticker
                result = pd.concat([result, df], ignore_index=True)

        else:
            print(f"Não foi possível obter os preços de fechamento para o ticker {ticker}.")

    return result

df_close = get_close(list(df_funds['ticker']))

# 4. Endereços dos imóveis dos FIIs

In [ ]:
def get_address(ticker_list):
    result = pd.DataFrame()

    for index, ticker in enumerate(ticker_list):
        # print(f'Obtendo endereços: {index+1}/{len(ticker_list)}')

        api_url = f'https://fii-api.infomoney.com.br/api/v1/propertie/{ticker}'
        response = requests.get(api_url)
        data = []

        if '{' in str(response.content):
            response = response.json()

            for item in response["property"]:
                row = {
                    "Ticker": ticker,
                    "Tipo": item["type"],
                    "Nome": item["name"],
                    "DataCompra": item["datePurchase"],
                    "ValorAreaBrutaLocavel": item["valueGrossLeasableArea"],
                    "Estado": item["state"],
                    "Cidade": item["city"],
                    "Endereco": item["address"],
                    "GoogleMapsLink": item["googleMapsLink"],
                    "PercentualPartic": item["percentagePartic"],
                    "PecentualVacancia": item["percentVacancy"],
                    "PercentualInadimplencia90Dias": item["percent90DayDeliquency"],
                    "PercentualFii": item["percentFii"],
                    "Latitude": float("NaN"),
                    "Longitude": float("NaN")
                }

                # Verifica se as coordenadas estão presentes no link do Google Maps
                cordinates = re.findall("(?<=@)[-]*[\d.]*,-[\d.]*", item['googleMapsLink'])

                if(len(cordinates) > 0):
                    cordinates = cordinates[0].split(",")
                    row["Latitude"], row["Longitude"] = float(cordinates[0]), float(cordinates[1])
                else:
                    # Se não estiverem presentes, faz uma solicitação à página do Google Maps para obter as coordenadas
                    address_url = ("https://www.google.com/maps/place/" + item["address"] + "," + item["city"] + "-" + item["state"]).replace(" ", "%20")
                    response = requests.get(address_url)

                    cordinates = re.findall("(?<=@)[-]*[\d.]*,-[\d.]*", str(response.content))

                    if(len(cordinates) > 0):
                        cordinates = cordinates[0].split(",")
                        row["Latitude"], row["Longitude"] = float(cordinates[0]), float(cordinates[1])

                data.append(row)

            df = pd.DataFrame(data)
            result = pd.concat([result, df], ignore_index=True)

    return result

df_address = get_address(list(df_funds['ticker']))

In [ ]:
percent = df_address['Latitude'].isnull().sum()/(len(df_address))*100
print(f'{len(df_address)} endereços de fundos imobiliários foram encontrados.')
print('%.2f%% dos endereços estão sem Latitude e Longitude.' % percent)

# 5. Índice IFIX

In [ ]:
def get_ifix():

    df_ifix = pd.DataFrame()

    headers = {
        'authority':'www.infomoney.com.br',
        'accept':'application/json, text/javascript, */*; q=0.01',
        'accept-language':'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
        'content-type':'application/x-www-form-urlencoded; charset=UTF-8',
        'origin':'https://www.infomoney.com.br',
        'referer':'https://www.infomoney.com.br/cotacoes/b3/indice/ifix/historico/',
    }

    body = f'page=0&numberItems=99999&initialDate={start_date}&finalDate={end_date}&symbol=IFIX'
    response = requests.post('https://www.infomoney.com.br/wp-json/infomoney/v1/quotes/history', headers=headers, data=body)

    if not str(response.content) == "b''":

        json_response = json.loads(response.content)

        jobject = []
        for obj in json_response:
            jobject.append({
                'data': obj[0]['display'],
                'Close': obj[2]
            })

        df_ifix = pd.DataFrame(jobject)
        df_ifix['Datetime'] = pd.to_datetime(df_ifix['data'], format='%d/%m/%Y')
        df_ifix.drop(columns={'data'}, inplace = True)

    return df_ifix

df_ifix = get_ifix()
df_ifix

,Close,Datetime
0,3.311,2023-12-28
1,3.299,2023-12-27
2,3.285,2023-12-26
3,3.260,2023-12-22
4,3.232,2023-12-21
...,...,...
493,2.760,2022-01-07
494,2.757,2022-01-06
495,2.760,2022-01-05
496,2.787,2022-01-04


# 6. Índices IGP-M, Selic e IPCA

In [ ]:
def obtem_dados_mercado():

    result = pd.DataFrame()
    indices = ['selic', 'ipca', 'igp-m']

    for indice in indices:

        # Obtém os dados do IGP-M
        response = requests.get(f'https://www.dadosdemercado.com.br/indices/{indice}')

        if response.status_code == 200:
            df = pd.read_html(response.content, encoding='utf-8')[0]

        anos = list(df.iloc[:, 0].values)

        timestamp = []
        values = []

        for i in range(len(anos)):
            for m in range(12, 0, -1):
                taxa = str(list(df.iloc[:, m].values)[i])
                if taxa != '--':
                    mes = str(m) if m > 9 else "0" + str(m)
                    timestamp.append(str(anos[i]) + "-" + mes)
                    values.append(round(float(taxa.replace("%","").replace(",",".")), 2))

        # Cria DataFrame
        df = pd.DataFrame({
            'Timestamp': timestamp,
            'Value': values,
            'Indice': indice.upper()
        })

        df['Value'] = pd.to_numeric(df['Value'], downcast="float")
        result = pd.concat([result, df], ignore_index=True)

    return result

# Exemplo de uso
df_indices = obtem_dados_mercado()
df_indices

# 7. Desdobramentos de FIIs

In [ ]:
# Função para converter o Ratio
def convert_ratio(ratio):
    left, right = ratio.split(':')
    return float(left) / float(right)

def get_splits():
    url = "https://br.investing.com/stock-split-calendar/Service/getCalendarFilteredData"
    headers = {
        "accept-language": "en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7,es;q=0.6,ar;q=0.5",
        "content-type": "application/x-www-form-urlencoded",
        "origin": "https://br.investing.com",
        "referer": "https://br.investing.com/stock-split-calendar/",
        "x-requested-with": "XMLHttpRequest"
    }
    data = {
        "country[]": "32",
        "dateFrom": "2019-01-01",
        "dateTo": "2024-01-01",
        "currentTab": "custom",
        "submitFilters": "1",
        "limit_from": "0"
    }

    response = requests.post(url, headers=headers, data=data)

    matches = re.finditer(r'\d{2}\.\d{2}\.\d{4}(.|\n)*?(?=\d{2}\.\d{2}\.\d{4}|$)', response.json()['data'])

    df_splits = pd.DataFrame()
    for match in matches:

        match_text = match.group(0)

        date = re.search(r'^(\d{2})\.(\d{2})\.(\d{4})', match_text)

        if date:

            matches2 = re.findall(r'(\d?[A-Z]{4}11B?)(?:.|\n)*?(\d+\:\d+)', match_text)
            df = pd.DataFrame(matches2, columns=['Ticker', 'Ratio'])
            df['Date'] = f'{date.group(3)}-{date.group(2)}-{date.group(1)}'
            df_splits = pd.concat([df_splits, df], ignore_index=True)

    df_splits['Ratio'] = df_splits['Ratio'].apply(convert_ratio)
    df_splits['Date'] = pd.to_datetime(df_splits['Date'])

    return df_splits

df_splits = get_splits()
df_splits

# 8. Enriquecimento de dados de imóveis - Google Maps

In [ ]:
def get_haversine_calculus(lat1, lon1, lat2, lon2):
    # Converter graus para radianos
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Raio médio da Terra em quilômetros
    radius = 6371

    # Diferença das coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula de Haversine
    a = math.sin(dlat/2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance * 1000

def convert_to_meters(valor):
    if valor.endswith('km'):
        valor = float(valor[:-2]) * 1000
    elif valor.endswith('m'):
        valor = float(valor[:-1])
    return str(int(valor))

def get_walking_distance(lat_origem, lon_origem, lat_destino, lon_destino):

    url = f"https://www.google.com/maps/dir/'{lat_destino},{lon_destino}'/{lat_origem},{lon_origem}/@{lat_destino},{lon_destino},17z/data=!3m1!4b1!4m7!4m6!1m3!2m2!1d{lon_destino}!2d{lat_destino}!1m0!3e2?entry=ttu"

    response = requests.get(url)
    content = response.content.decode('utf-8')

    pattern = r'\[[\d.]+,\\"([\d.]+\s+(?:km|m))\\",[\d.]+\],\[[\d.]+,\\"[\d.]+\s+(?:min|sec|hr)'
    result = re.findall(pattern, content)

    if result:
        return convert_to_meters(result[0])

    return -1

def get_targets(target, latitude, longitude, radius):

    url = f'https://www.google.com.br/maps/search/{target}/@{latitude},{longitude},15z/data=!3m1!4b1?entry=ttu'

    response = requests.get(url)
    content = response.content.decode('utf-8')
    content = content.replace('\\\\u0026','&')

    pattern_without_rating = r'((?:-|\+)\d+\.\d+),((?:-|\+)\d+\.\d+)\],\\"0x\w+?:0x\w+?\\",\\\"(.*?)\\"'
    pattern_with_rating = r'([012345]{1}\.\d{1}),(\d+)\](?:,.*?){7}((?:-|\+)\d+\.\d+),((?:-|\+)\d+\.\d+)\],\\"0x\w+?:0x\w+?\\",\\\"(.*?)\\"'

    match_without_rating = re.findall(pattern_without_rating, content)
    match_with_rating = re.findall(pattern_with_rating, content)

    result = []
    for m in match_with_rating + match_without_rating:

        is_duplicate = any(item['nome'] == m[-1] for item in result)

        if not is_duplicate:

            euclidean_distance = get_haversine_calculus(latitude, longitude, float(m[-3]), float(m[-2]))

            if euclidean_distance < radius:

                result.append({
                    'rating': float(m[0]) if len(m) == 5 else 0,
                    'evaluators': int(m[1]) if len(m) == 5 else 0,
                    'latitude': float(m[-3]),
                    'longitude': float(m[-2]),
                    'walking_distance': get_walking_distance(latitude, longitude, float(m[-3]), float(m[-2])),
                    'euclidean_distance': euclidean_distance,
                    'nome': m[-1]
                })

    df_result = pd.DataFrame(result)

    if len(df_result) > 1:

        df_result['walking_distance'] = df_result['walking_distance'].astype(int)

        df_result = df_result[df_result['walking_distance'] <= radius]

        df_result = df_result.sort_values('walking_distance')

    return df_result

In [ ]:
df_adress_enhanced = df_address.copy()

for index, row in df_adress_enhanced.iterrows():

    print(f"{index+1}/{len(df_adress_enhanced)}...")

    if isinstance(row['Latitude'], (int, float)) and math.isnan(row['Latitude']):
        continue

    if isinstance(row['Longitude'], (int, float)) and math.isnan(row['Longitude']):
        continue

    targets = ["restaurante", "farmácia", "academia", "escola", "supermercado", "posto policial", "hotel", "estação de metrô"]

    for target in targets:

        try:
            df_adress_enhanced_tt = get_targets(target, row['Latitude'], row['Longitude'], 1000)
        except Exception as err:
            df_adress_enhanced_tt = pd.DataFrame()
            print(f"Unexpected {err=}, {type(err)=}")

        weighted_mean = 0

        if len(df_adress_enhanced_tt) > 0:

            # Calcule a média ponderada
            weighted_mean = (df_adress_enhanced_tt['rating'] * df_adress_enhanced_tt['evaluators']).sum() / df_adress_enhanced_tt['evaluators'].sum()

        df_adress_enhanced.at[index, f"{target.replace(' ','_')}_average_rating"] = weighted_mean
        df_adress_enhanced.at[index, f"{target.replace(' ','_')}_total"] = len(df_adress_enhanced_tt)

In [ ]:
df_adress_enhanced

# 9. Pré-processamento dos dados

In [ ]:
df_close.rename(columns={'valor': 'Close', 'ticker': 'Ticker', 'data': 'Datetime'}, inplace = True)
df_close['Ticker'] = df_close['Ticker'].astype('category')
df_close['Datetime'] = pd.to_datetime(df_close['Datetime'], format='%d-%m-%YT%H:%M:%S')

In [ ]:
df_dividends = df_dividends[['ticker', 'data_pagamento', 'valor']].copy()
df_dividends.rename(columns={'data_pagamento': 'Datetime', 'ticker': 'Ticker', 'valor': 'Dividends'}, inplace=True)
df_dividends['Ticker'] = df_dividends['Ticker'].astype('category')
df_dividends['Datetime'] = pd.to_datetime(df_dividends['Datetime'], format='%Y-%m-%d', errors='coerce')
df_dividends = df_dividends.dropna(subset=['Datetime'])
df_dividends = df_dividends[(df_dividends['Datetime'] >= start_date) & (df_dividends['Datetime'] < end_date)]

In [ ]:
def is_year_month_present(timestamps, year_month):
    year_months = [ts.strftime("%Y-%m") for ts in timestamps]

    return year_month in year_months

def has_missing_data(df_history):

    min_date = df_history['Datetime'].min()
    max_date = df_history['Datetime'].max()

    start_date = pd.to_datetime(max_date).replace(day=1, hour=0, minute=0, second=0, microsecond=0)
    end_date = pd.to_datetime(min_date).replace(day=1, hour=0, minute=0, second=0, microsecond=0)

    while start_date >= end_date:
        year_month = start_date.strftime("%Y-%m")

        if not is_year_month_present(list(df_history['Datetime']), year_month):
            x = list(df_history['Datetime'])
            print(f'{year_month} - {x}')
            return True

        start_date -= relativedelta(months=1)

    return False

def get_month_close(df_close, date):

    date = pd.to_datetime(date)

    year = date.year
    month = date.month

    start_date = pd.to_datetime('today').replace(year=year, month= month, day=1,hour=0,minute=0,second=0,microsecond=0)
    end_date = (start_date + relativedelta(months=1))

    df_aux = df_close.copy()

    df_aux = df_aux[df_aux['Datetime'] >= start_date]
    df_aux = df_aux[df_aux['Datetime'] < end_date]

    if len(df_aux) > 0:
        return float(df_aux.iloc[-1]['Close'])
    else:
        return -1

def get_history(df_close, df_dividends, fund, years):

    df_close_aux = df_close[df_close['Ticker'] == fund]
    df_history = df_dividends[df_dividends['Ticker'] == fund].copy()

    if len(df_history) > 0 and len(df_close_aux) > 0:

        new_df = []
        for index, row in df_history.iterrows():

            row['Dividends'] = round(row['Dividends'],2)
            row['Close'] = get_month_close(df_close_aux, row['Datetime'])
            new_df.append(row)

        df_history = pd.DataFrame(new_df)

        datas = list(df_history['Datetime'])

        if has_missing_data(df_history):
            print("FII " + fund + " será removido por estar com dados faltantes.")
            df_history = pd.DataFrame()

    return df_history

def ajusta_desdobramento(df, df_splits):
    df_splits['Date'] = pd.to_datetime(df_splits['Date'])  # Convertendo a coluna 'Date' para datetime

    for index, row in df.iterrows():
        ticker = row['Ticker']
        date = row['Datetime']
        close_price = row['Close']

        desdobramento = df_splits[df_splits['Ticker'] == ticker]

        if not desdobramento.empty:
            for _, desd_row in desdobramento.iterrows():
                desd_date = desd_row['Date']

                # Extraindo o ano e o mês da data a ser comparada
                date_year_month = date.to_period('M')

                # Extraindo o ano e o mês da data em desd_row
                desd_year_month = desd_date.to_period('M')

                # Verificando se o ano e o mês de `date` são menores do que o ano e o mês de `desd_date`
                if date_year_month < desd_year_month:
                    ratio = desd_row['Ratio']
                    df.at[index, 'Close'] = round(close_price / ratio, 2)

    return df

def remove_big_variations(df, col_dict):
    drop_indexes = []
    for fund in df['Ticker'].unique():
        df_variacoes = df[df['Ticker'] == fund].sort_values(by='Datetime', ascending=True)

        for key in col_dict:
            df_variacoes = df_variacoes[(abs(df_variacoes[key]) >= col_dict[key])]

            if len(df_variacoes) > 0:
                drop_indexes = drop_indexes + list(df[(df["Datetime"] <= df_variacoes["Datetime"].values[-1]) & (df.Ticker == fund)].index)

    df = df.drop(drop_indexes)
    ticker_before = df['Ticker'].unique()

    # Remove funds que não possuem pelo menos 20 registros
    for fund in df['Ticker'].unique():
        if len(df[df['Ticker'] == fund]) < 20:
            df = df[df['Ticker'] != fund]

    removed_tickers = set(ticker_before) - set(df['Ticker'].unique())

    if(len(removed_tickers) > 0):
        print("Tickers removidos: ", removed_tickers)
    else:
        print("Nenhum ticker foi removido")

    return df

def add_pct_changes(df_close):
    # Cria o DataFrame a ser aprimorado
    df_improved = df_close.copy()

    # Remove funds que não possuem pelo menos 12 registros
    for fund in df_improved['Ticker'].unique():
        if len(df_improved[df_improved['Ticker'] == fund]) < 12:
            df_improved = df_improved[df_improved['Ticker'] != fund]

    # Normaliza os dados que sofreram desdobramento
    ajusta_desdobramento(df_improved, df_splits)
    df_improved = df_improved.replace(np.inf, 0).replace(-np.inf, 0).replace(0, 0.001)

    # Cria a coluna DividendYield
    df_improved['DividendYield'] = round(100 * df_improved['Dividends'] / df_improved['Close'], 6)

    # Cria novas colunas contendo a variação de valores ao longo dos meses
    for fund in df_improved['Ticker'].unique():

        print("Incrementando informações de " + fund + "...")

        df_fund = df_improved[df_improved.Ticker == fund].sort_values(by='Datetime', ascending=True)
        df_improved.loc[df_improved.Ticker == fund, 'DividendsChange'] = round(df_fund['Dividends'].pct_change(), 6)
        df_improved.loc[df_improved.Ticker == fund, 'CloseChange'] = round(df_fund['Close'].pct_change(), 6)
        df_improved.loc[df_improved.Ticker == fund, 'DividendYieldChange'] = round(df_fund['DividendYield'].pct_change(), 6)
        df_improved.loc[df_improved.Ticker == fund, 'DividendsChange6M'] = round(df_fund['Dividends'].pct_change(periods=6), 6)
        df_improved.loc[df_improved.Ticker == fund, 'CloseChange6M'] = round(df_fund['Close'].pct_change(periods=6), 6)
        df_improved.loc[df_improved.Ticker == fund, 'DividendYieldChange6M'] = round(df_fund['DividendYield'].pct_change(periods=6), 6)

    # Procura no DataFrame registros com variações muito discrepantes
    df_improved = remove_big_variations(df_improved, {'DividendsChange': 500, 'CloseChange': 0.35})

    return df_improved

def process_history(df_funds, years):

    df_history = pd.DataFrame()

    # Percorre a lista de funds para obter o histórico individual de cada um deles
    for fund in df_funds['ticker']:

        print("Coletando informações de " + fund + "...")

        df_aux = get_history(df_close, df_dividends, fund, years)
        df_history = pd.concat([df_history, df_aux], ignore_index=True)

        print(str(len(df_aux)) + " dados de histórico foram encontrados.")

    if df_history.empty:
        print("O DataFrame está vazio.")
        return df_history

    # Tratar valores NaN
    is_NaN = df_history.isnull()
    row_has_NaN = is_NaN.any(axis=1)
    rows_with_NaN = df_history[row_has_NaN]

    if not rows_with_NaN.empty:
        tickers = rows_with_NaN['Ticker'].unique()
        df_history = df_history[~df_history['Ticker'].isin(tickers)]

    # Remover duplicatas e substituir valores infinitos e zero
    df_history = df_history.drop_duplicates().replace([np.inf, -np.inf, 0], 0.001)

    # Verificar a quantidade de registros por fund
    for fund in df_history['Ticker'].unique():
        if len(df_history[df_history['Ticker'] == fund]) < 12:
            df_history = df_history[df_history['Ticker'] != fund]

    # Comparar tickers no último mês
    if 'ticker' in df_history.columns and 'Datetime' in df_history.columns:

        target_date = pd.to_datetime(end_date, format='%d-%m-%Y')

        # Extraindo o ano e o mês de end_date
        target_year = target_date.year
        target_month = target_date.month

        # Filtre os valores que correspondem ao mês e ano desejados
        a = df_history[(df_history['Datetime'].dt.year == target_year) & (df_history['Datetime'].dt.month == target_month)]
        b = df_history.ticker.unique()
        intersection = list(set(a) & set(b))
        funds_faltantes = list(set(a) ^ set(b))

        df_history = df_history[~df_history['Ticker'].isin(funds_faltantes)]

    return df_history

def add_indices(df_history):
    # Convertendo colunas de datas para datetime
    df_ifix['Datetime'] = pd.to_datetime(df_ifix['Datetime'])
    df_indices['Timestamp'] = pd.to_datetime(df_indices['Timestamp'], format='%Y-%m')

    # Criando colunas YYYY-MM nos DataFrames
    df_ifix['YYYY-MM'] = df_ifix['Datetime'].dt.to_period('M')
    df_indices['YYYY-MM'] = df_indices['Timestamp'].dt.to_period('M')
    df_history['YYYY-MM'] = df_history['Datetime'].dt.to_period('M')

    # Obter o último valor de df_ifix para cada YYYY-MM
    # Obter o último valor de df_ifix para cada YYYY-MM
    df_ifix_last = df_ifix.sort_values(by='Datetime').groupby('YYYY-MM').last().reset_index()

    # Renomeando coluna Close para IFIX antes da união
    df_ifix_last.rename(columns={'Close': 'IFIX'}, inplace=True)

    # Pivotando df_indices para ter uma coluna para cada índice
    df_indices_pivot = df_indices.pivot(index='YYYY-MM', columns='Indice', values='Value').reset_index()

    # Unindo df_history com df_ifix_last
    df_history = df_history.merge(df_ifix_last[['YYYY-MM', 'IFIX']], on='YYYY-MM', how='left')

    # Unindo df_history com df_indices_pivot
    df_history = df_history.merge(df_indices_pivot, on='YYYY-MM', how='left')

    # Removendo coluna temporária YYYY-MM
    df_history.drop(columns=['YYYY-MM'], inplace=True)

    df_history.rename(columns={'IGP-M': 'IGPM'}, inplace=True)

    return df_history

def getSectorMeans(df_funds, df_history):

    df_setores = []

    for setor in df_funds["setor"].unique():
        setor_tickers = df_funds[df_funds["setor"] == setor]["ticker"].values
        df_sector = df_history[df_history["Ticker"].isin(setor_tickers)]

        min_date = pd.to_datetime(df_sector["Datetime"].min()).replace(day=1)
        max_date = pd.to_datetime(df_sector["Datetime"].max()).replace(day=1)

        while min_date <= max_date:
            date = min_date.strftime("%Y-%m")

            setor_data = {
                'setor': setor,
                'Datetime': date,
                'DividendsChangeMean': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["DividendsChange"].mean(),
                'CloseChangeMean': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["CloseChange"].mean(),
                'DividendYieldChangeMean': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["DividendYieldChange"].mean(),
                'DividendsChangeMean6M': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["DividendsChange6M"].mean(),
                'CloseChangeMean6M': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["CloseChange6M"].mean(),
                'DividendYieldChangeMean6M': df_sector[df_sector["Datetime"].dt.strftime("%Y-%m") == date]["DividendYieldChange6M"].mean(),
            }

            df_setores.append(setor_data)
            min_date += relativedelta(months=1)

    return pd.DataFrame(df_setores)


def add_sector_pct_changes(df_funds, df_history):

    df_sectors = getSectorMeans(df_funds, df_history)

    # Insere preço dos índices e a média do setor ao longo do tempo
    for index, fundo in enumerate(df_history['Ticker'].unique()):

        print(str(index+1) + "/" + str(len(df_history['Ticker'].unique())))

        sector = df_funds[df_funds["ticker"] == fundo]["setor"].values[0]

        for data in df_history['Datetime']:

            sector_values = df_sectors[(df_sectors["Datetime"] == data.strftime("%Y-%m")) & (df_sectors["setor"] == sector)]

            if len(sector_values) > 0:
                for mean_col in ["DividendsChangeMean", "CloseChangeMean", "DividendYieldChangeMean"]:
                    mean_value = sector_values[mean_col].values[0]
                    df_history.loc[(df_history.Ticker == fundo) & (df_history.Datetime == data), "Sector" + mean_col] = float(mean_value)

    return df_history

In [ ]:
df_history = process_history(df_funds, 2)

Coletando informações de AAGR11...
0 dados de histórico foram encontrados.
Coletando informações de AAZQ11...
11 dados de histórico foram encontrados.
Coletando informações de ABCP11...
2023-01 - [Timestamp('2023-12-07 00:00:00'), Timestamp('2023-11-08 00:00:00'), Timestamp('2023-10-06 00:00:00'), Timestamp('2023-09-08 00:00:00'), Timestamp('2023-08-07 00:00:00'), Timestamp('2023-07-07 00:00:00'), Timestamp('2023-06-07 00:00:00'), Timestamp('2023-05-08 00:00:00'), Timestamp('2023-04-10 00:00:00'), Timestamp('2023-03-07 00:00:00'), Timestamp('2023-02-07 00:00:00'), Timestamp('2022-12-07 00:00:00'), Timestamp('2022-11-08 00:00:00'), Timestamp('2022-10-07 00:00:00'), Timestamp('2022-09-08 00:00:00'), Timestamp('2022-08-05 00:00:00'), Timestamp('2022-07-07 00:00:00'), Timestamp('2022-06-07 00:00:00'), Timestamp('2022-05-06 00:00:00'), Timestamp('2022-04-07 00:00:00'), Timestamp('2022-03-08 00:00:00'), Timestamp('2022-02-07 00:00:00'), Timestamp('2022-01-10 00:00:00'), Timestamp('2021-12-08

In [ ]:
df_history = add_pct_changes(df_history)

Incrementando informações de AFHI11...
Incrementando informações de AIEC11...
Incrementando informações de ALZR11...
Incrementando informações de ARCT11...
Incrementando informações de ARRI11...
Incrementando informações de BARI11...
Incrementando informações de BBFI11B...
Incrementando informações de BBFO11...
Incrementando informações de BBIM11...
Incrementando informações de BBPO11...
Incrementando informações de BBRC11...
Incrementando informações de BCFF11...
Incrementando informações de BCIA11...
Incrementando informações de BCRI11...
Incrementando informações de BICR11...
Incrementando informações de BIME11...
Incrementando informações de BLCP11...
Incrementando informações de BLMC11...
Incrementando informações de BLMG11...
Incrementando informações de BLMO11...
Incrementando informações de BLMR11...
Incrementando informações de BMLC11...
Incrementando informações de BNFS11...
Incrementando informações de BPFF11...
Incrementando informações de BPRP11...
Incrementando informaçõe

In [ ]:
df_history = add_indices(df_history)

In [ ]:
df_history = add_sector_pct_changes(df_funds, df_history)

1/218
2/218
3/218
4/218
5/218
6/218
7/218
8/218
9/218
10/218
11/218
12/218
13/218
14/218
15/218
16/218
17/218
18/218
19/218
20/218
21/218
22/218
23/218
24/218
25/218
26/218
27/218
28/218
29/218
30/218
31/218
32/218
33/218
34/218
35/218
36/218
37/218
38/218
39/218
40/218
41/218
42/218
43/218
44/218
45/218
46/218
47/218
48/218
49/218
50/218
51/218
52/218
53/218
54/218
55/218
56/218
57/218
58/218
59/218
60/218
61/218
62/218
63/218
64/218
65/218
66/218
67/218
68/218
69/218
70/218
71/218
72/218
73/218
74/218
75/218
76/218
77/218
78/218
79/218
80/218
81/218
82/218
83/218
84/218
85/218
86/218
87/218
88/218
89/218
90/218
91/218
92/218
93/218
94/218
95/218
96/218
97/218
98/218
99/218
100/218
101/218
102/218
103/218
104/218
105/218
106/218
107/218
108/218
109/218
110/218
111/218
112/218
113/218
114/218
115/218
116/218
117/218
118/218
119/218
120/218
121/218
122/218
123/218
124/218
125/218
126/218
127/218
128/218
129/218
130/218
131/218
132/218
133/218
134/218
135/218
136/218
137/218
138/218
139/

In [ ]:
df_history['Datetime'] = pd.to_datetime(df_history['Datetime']).dt.to_period('M')
df_history = df_history.dropna(subset=['DividendsChange'])
df_history

,Ticker,Datetime,Dividends,Close,DividendYield,DividendsChange,CloseChange,DividendYieldChange,DividendsChange6M,CloseChange6M,DividendYieldChange6M,IFIX,IGPM,IPCA,SELIC,SectorDividendsChangeMean,SectorCloseChangeMean,SectorDividendYieldChangeMean
0,AFHI11,2023-12,1.00,100.00,1.000000,-0.019608,0.049428,-0.065785,-0.090909,0.030928,-0.118182,3.311,0.74,0.56,12.15,1.519198,0.027347,1.438945
1,AFHI11,2023-11,1.02,95.29,1.070417,-0.028571,-0.003555,-0.025105,-0.072727,0.003898,-0.076328,3.177,0.59,0.28,12.65,0.100949,-0.008095,0.104989
2,AFHI11,2023-10,1.05,95.63,1.097982,0.000000,-0.005718,0.005752,-0.045455,0.034397,-0.077196,3.156,0.50,0.24,12.65,1.195540,-0.013429,1.482554
3,AFHI11,2023-09,1.05,96.18,1.091703,0.000000,-0.011511,0.011644,-0.009434,0.037876,-0.045584,3.219,0.37,0.26,13.15,2.097184,-0.013467,2.110365
4,AFHI11,2023-08,1.05,97.30,1.079137,-0.045455,0.011960,-0.056736,0.000000,0.036540,-0.035251,3.213,-0.14,0.23,13.65,0.124187,-0.011855,0.120791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7546,XPSF11,2021-06,0.09,9.39,0.958466,0.125000,-0.043788,0.176517,NaN,NaN,NaN,NaN,0.60,0.53,3.40,-0.000547,-0.047392,0.048682
7547,XPSF11,2021-05,0.08,9.82,0.814664,0.142857,0.005118,0.137038,NaN,NaN,NaN,NaN,4.10,0.83,2.65,0.003464,-0.024275,0.030408
7548,XPSF11,2021-04,0.07,9.77,0.716479,-0.125000,0.008256,-0.132164,NaN,NaN,NaN,NaN,1.51,0.31,2.65,0.063068,-0.003389,0.069827
7549,XPSF11,2021-03,0.08,9.69,0.825593,0.142857,-0.005133,0.148753,NaN,NaN,NaN,NaN,2.94,0.93,1.90,0.048823,-0.022867,0.072185
